In [2]:
:opt no-lint

In [3]:
import Test.QuickCheck
import Test.QuickCheck.Checkers
import Test.QuickCheck.Classes

#  21 Traversable
## 21.1 Traversable
`Traversable` allows to traverse a data structure while transforming its elements like a functor, producing applicative effects along the way, and lift those potentially multiple instances of applicative structure outside of the traversable structure.

## 21.2 The Traversable type class definition
`Traversable` depends on `Applicative`, and thus `Functor`, and is superclassed by `Foldable`.
It is defined by at least one of the functions:
- `traverse` which maps each element of a structure to an action, evaluates the actions from left to right, and collects the results.
- `sequenceA` which evaluates each action in the structure from left to right, and collect the results.
The caller of such functions chooses the `Applicative` instance. 

## 21.3 sequenceA
The effect of `sequenceA` is to flip two structures.

#### Example:

In [43]:
sequenceA $ Just [1,2,3]
sequenceA [Just 1, Just 2, Just 3]

[Just 1,Just 2,Just 3]

Just [1,2,3]

## 21.4 traverse
Features:
- maps a function over some embedded value(s), like `fmap`
- generates more structure, likie `=<<`
- moreover, the additional structure can be of a different type
- at the end, it will flip the two structures around, as `sequenceA` does

#### Example:

In [44]:
traverse Just [1, 2, 3]

Just [1,2,3]

### mapM is traverse
`traverse` can be seen as an abstraction of the `[]` in `mapM` to any traversable data structure and generalizing the `Monad` requirement to only need an `Applicative`.
## 21.5 So, what’s Traversable for?
- to flip two type constructors
- to map something and then flip them around
- instead of `sequence` or `sequenceA` combined with a `map` or `fmap`
- to fold a structure while producing effects

## 21.6 Morse code revisited
We can use `traverse` to map a computation that can potentially fail to a traversable structure and we what to result in a `Nothing` if at least one element lead the computation to fail.
`traverse` is `sequence` (`Traversable` distinctive feature) composed with `fmap`, in the same way as `>>=` is `join` (`Monad` distinctive feature) composed with `fmap`.
## 21.7 Axing tedious code
The section shows how to use `traverse` to refactor a function where we need to map over a function while performing actions and combine the results.
## 21.8 Do all the things
The example shows how to use `traverse` in order to visit a list of URLs and return a single `IO` action containing the list of responses.
### Strength for understanding
The `Functor` instance can be recovered from the `Traversable` instance as follow:

In [45]:
import Data.Functor.Identity

fmap' f t = runIdentity $ traverse (Identity . f) t
fmap' (+1) [1, 2, 3]

[2,3,4]

The `Foldable` instance can be recovered from the `Traversable` instance as follow:

In [46]:
import Data.Functor.Constant
import Data.Monoid

foldMap' f t = getConstant $ traverse (Constant . f) t
getSum $ foldMap' Sum [1, 2, 3]

6

## 21.9 Traversable instances
The following two instances only requires `Functor`, thus underutilizing the `Traversable` constraints.
### Either
Produces an `Either` context inside an applicative context.
### Tuple
Produces an `Tuple` context inside an applicative context.
## 21.10 Traversable laws
#### `traverse` laws:
1. Naturality: `t . traverse f = traverse (t . f)`
2. Identity: `traverse Identity = Identity`
3. Composition: `traverse (Compose . fmap g . f) = Compose . fmap (traverse g) . traverse f`

#### `sequenceA` laws:
1. Naturality: `t . sequenceA = sequenceA . fmap t`
2. Identity: `sequenceA . fmap Identity = Identity`
3. Composition: `sequenceA . fmap Compose = Compose . fmap sequenceA . sequenceA`

## 21.11 Quality control
The `Traversable` laws can be checked with `QuickCheck` and `checkers` libraries.
## 21.12 Chapter exercises
Traversable instances
Write a `Traversable` instance and validate it with `QuickCheck`
#### Identity

In [47]:
newtype Identity a = Identity a deriving (Eq, Ord, Show)

instance Functor Identity where
  fmap f (Identity x) = Identity $ f x
  
instance Foldable Identity where
  foldMap f (Identity x) = f x

instance Traversable Identity where
  sequenceA (Identity x) = Identity <$> x
  traverse f (Identity x) = Identity <$> f x
  
instance (Arbitrary a) => Arbitrary (Identity a) where
  arbitrary = Identity <$> arbitrary
      
instance (Eq a) => EqProp (Identity a) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Identity Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Constant

In [82]:
newtype Constant a b = Constant { getConstant :: a } deriving (Eq, Show)

instance Functor (Constant a) where
  fmap _ (Constant x) = Constant x

instance Foldable (Constant a) where
  foldMap _ _ = mempty

instance Traversable (Constant a) where
  sequenceA (Constant x) = Constant <$> pure x
  traverse _ (Constant x) = Constant <$> pure x

instance (Arbitrary a, Arbitrary b) => Arbitrary (Constant a b) where
  arbitrary = Constant <$> arbitrary

instance (Eq a, Eq b) => EqProp (Constant a b) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Constant Trigger Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Maybe

In [49]:
data Optional a = Nada | Yep a deriving (Eq, Show)

instance Functor Optional where
  fmap _ Nada = Nada
  fmap f (Yep x) = Yep $ f x

instance Foldable Optional where
  foldMap _ Nada = mempty
  foldMap f (Yep x) = f x

instance Traversable Optional where
  sequenceA Nada = pure Nada
  sequenceA (Yep x) = Yep <$> x
  traverse _ Nada = pure Nada
  traverse f (Yep x) = Yep <$> f x

instance (Arbitrary a) => Arbitrary (Optional a) where
  arbitrary = do
    x <- arbitrary
    elements [Nada, Yep x]

instance (Eq a) => EqProp (Optional a) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Optional Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### List

In [85]:
data List a = Nil | Cons a (List a) deriving (Eq, Show)

instance Functor List where
  fmap _ Nil = Nil
  fmap f (Cons x xs) = Cons (f x) (fmap f xs)

instance Foldable List where
  foldMap _ Nil = mempty
  foldMap f (Cons x xs) = f x <> foldMap f xs

instance Traversable List where
  sequenceA Nil = pure Nil
  sequenceA (Cons x xs) = Cons <$> x <*>  sequenceA xs
  traverse _ Nil = pure Nil
  traverse f (Cons x xs) = Cons <$> f x <*>  traverse f xs

instance (Arbitrary a) => Arbitrary (List a) where
  arbitrary =  do
  a <- arbitrary
  b <- arbitrary
  frequency [ (10, return $ Cons a b), (1, return Nil)]

instance Eq a => EqProp (List a) where
  xs =-= ys = xs' `eq` ys'
    where
      xs' = let l = xs in take' 100 l
      ys' = let l = ys in take' 100 l

take' :: Int -> List a -> List a
take' 0 _ = Nil
take' _ Nil = Nil
take' n (Cons x xs) = Cons x (take' (n - 1) xs)

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: List Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Three

In [51]:
data Three a b c = Three a b c deriving (Eq, Show)

instance Functor (Three a b) where
  fmap f (Three x y z) = Three x y (f z)

instance Foldable (Three a b) where
  foldMap f (Three _ _ x) = f x

instance Traversable (Three a b) where
  sequenceA (Three x y z) = Three x y <$> z
  traverse f (Three x y z) = Three x y <$> f z

instance (Arbitrary a, Arbitrary b, Arbitrary c) => Arbitrary (Three a b c) where
  arbitrary = Three <$> arbitrary <*> arbitrary <*> arbitrary

instance (Eq a, Eq b, Eq c) => EqProp (Three a b c) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Three Trigger Trigger Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Pair

In [80]:
data Pair a b = Pair a b deriving (Eq, Show)

instance Functor (Pair a) where
  fmap f (Pair x y) = Pair x (f y)

instance Foldable (Pair a) where
  foldMap f (Pair _ x) = f x

instance Traversable (Pair a) where
  sequenceA (Pair x y) = Pair x <$> y
  traverse f (Pair x y) = Pair x <$> f y

instance (Arbitrary a, Arbitrary b) => Arbitrary (Pair a b) where
  arbitrary = Pair <$> arbitrary <*> arbitrary

instance (Eq a, Eq b) => EqProp (Pair a b) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Three Trigger Trigger Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Big

In [53]:
data Big a b = Big a b b deriving (Eq, Show)

instance Functor (Big a) where
  fmap f (Big x y z) = Big x (f y) (f z)

instance Foldable (Big a) where
  foldMap f (Big _ y z) = f y <> f z

instance Traversable (Big a) where
  sequenceA (Big x y z) = Big x <$> y <*> z
  traverse f (Big x y z) = Big x <$> f y <*> f z

instance (Arbitrary a, Arbitrary b) => Arbitrary (Big a b) where
  arbitrary = Big <$> arbitrary <*> arbitrary <*> arbitrary

instance (Eq a, Eq b) => EqProp (Big a b) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Big Trigger Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Bigger

In [54]:
data Bigger a b = Bigger a b b b  deriving (Eq, Show)

instance Functor (Bigger a) where
  fmap f (Bigger x y z t) = Bigger x (f y) (f z) (f t)

instance Foldable (Bigger a) where
  foldMap f (Bigger _ y z t) = f y <> f z <> f t

instance Traversable (Bigger a) where
  sequenceA (Bigger x y z t) = Bigger x <$> y <*> z <*> t
  traverse f (Bigger x y z t) = Bigger x <$> f y <*> f z <*> f t

instance (Arbitrary a, Arbitrary b) => Arbitrary (Bigger a b) where
  arbitrary = Bigger <$> arbitrary <*> arbitrary <*> arbitrary  <*> arbitrary

instance (Eq a, Eq b) => EqProp (Bigger a b) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Bigger Trigger Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### S

In [4]:
{-# LANGUAGE FlexibleContexts #-}

data S n a = S (n a) a deriving (Eq, Show)

instance Functor n => Functor (S n) where
  fmap f (S na x) = S (f <$> na) (f x)

instance Foldable n => Foldable (S n) where
  foldMap f (S na x) = foldMap f na <> f x

instance Traversable n => Traversable (S n) where
  sequenceA (S na x) = S <$> sequenceA na <*> x
  traverse f (S na x) = S <$> traverse f na <*> f x

instance (Functor n, Arbitrary (n a), Arbitrary a) => Arbitrary (S n a) where
  arbitrary = S <$> arbitrary <*> arbitrary

instance (Applicative n, Testable (n Property), Eq a, Eq (n a), EqProp a)=> EqProp (S n a) where
  (=-=) = eq

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: S Maybe Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.

#### Instances for Tree
This might be hard. Write the following instances for Tree:

In [86]:
data Tree a = Empty | Leaf a | Node (Tree a) a (Tree a) deriving (Eq, Show)

instance Functor Tree where
  fmap _ Empty = Empty
  fmap f (Leaf x) = Leaf $ f x
  fmap f (Node ls x rs) = Node (f <$> ls) (f x) (f <$> rs)

instance Foldable Tree where
  foldMap _ Empty = mempty
  foldMap f (Leaf x) = f x
  foldMap f (Node ls x rs) = foldMap f ls <> f x <> foldMap f rs
  
  foldr _ z Empty = z
  foldr f z (Leaf x) = f x z
  foldr f z (Node ls x rs) = f x (foldr f (foldr f z rs) ls)

instance Traversable Tree where
  sequenceA Empty = pure Empty
  sequenceA (Leaf x) = Leaf <$> x
  sequenceA (Node ls x rs) = Node <$> sequenceA ls <*> x <*>  sequenceA rs
  
  traverse _ Empty = pure Empty
  traverse f (Leaf x) = Leaf <$> f x
  traverse f (Node ls x rs) = Node <$> traverse f ls <*> f x <*>  traverse f rs

instance (Arbitrary a) => Arbitrary (Tree a) where
  arbitrary = do
    ls <- arbitrary
    x <- arbitrary
    rs <- arbitrary
    elements [ Empty, Leaf x, Node ls x rs]

instance Eq a => EqProp (Tree a) where
  xs =-= ys = xs' `eq` ys'
    where
      xs' = let l = xs in take' 100 l
      ys' = let l = ys in take' 100 l

take' :: Int -> Tree a -> Tree a
take' 0 _ = Empty
take' _ Empty = Empty
take' _ (Leaf x) = Leaf x
take' n (Node ls x rs) = Node (take' (n - 1) ls) x (take' (n - 1) rs)

type Trigger = (Int, Int, [Int])

do
  let trigger = undefined
  quickBatch $ traversable (trigger :: Tree Trigger)


traversable:
  fmap:    +++ OK, passed 500 tests.
  foldMap: +++ OK, passed 500 tests.